In [2]:
# In TensorFlow once you import the library, commands
# such as tf.constant create new nodes in a global
# "default graph". Then for example the tf.matmul
# command actually creates a new node whose value is
# tied by an equation to the values of c,d. When evaluated
# in a Session via sess.run, the values of these derived
# nodes can be extracted.

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np

from tensorflow.python.ops import array_ops

# Build a dataflow graph.
c = tf.constant([[1.0, 2.0], [3.0, 4.0]])
d = tf.constant([[1.0, 1.0], [0.0, 1.0]])
e = tf.matmul(c, d)
x = tf.constant([[1.0, 2.0, 3.0]])
t = tf.matmul(tf.transpose(x),x)
t = tf.reshape(t,[-1])

# Construct a `Session` to execute the graph.
sess = tf.Session()

# Execute the graph and store the value that `e` represents in `result`.
result = sess.run(t)
print(result)

[ 1.  2.  3.  2.  4.  6.  3.  6.  9.]


In [2]:
# Here is an example of the main loop in our linear logic RNN
# the batch size is 2, input size is 1, state size is 2

inputs = tf.constant([[0.0],[1.0]])
state = tf.constant([[1.0, 2.0],[3.0, 4.0]])
h = array_ops.concat(1, [inputs,state])

# Now the row index in inputs and shape currently represents the batch
# we convert this to a list index using unpack, after first inserting
# a new index to be the row
h2 = tf.expand_dims(h,1)
h3 = tf.unpack(h2) # a list, containing 1x3 row vectors
li = []
for row in h3:
    li.append(tf.matmul(tf.transpose(row),row))
h4 = tf.pack(li)
h5 = tf.reshape(h4,[int(h4.get_shape()[0]),-1]) # convert back to a 2D tensor

print("inputs shape: " + str(inputs.get_shape()))
print("inputs is: ")
print(sess.run(inputs))
print("state shape: " + str(state.get_shape()))
print("state is: ")
print(sess.run(state))
print("h shape: " + str(h.get_shape()))
print("h is: ")
print(sess.run(h))
print("h2 shape: " + str(h2.get_shape()))
print("first element of h3 shape: " + str(h3[0].get_shape()))
print("first element of h3 is: ")
print(sess.run(h3[0]))
print("h4 shape: " + str(h4.get_shape()))
print(sess.run(h4))
print("h5 shape: " + str(h5.get_shape()))
print(sess.run(h5))

inputs shape: (2, 1)
inputs is: 
[[ 0.]
 [ 1.]]
state shape: (2, 2)
state is: 
[[ 1.  2.]
 [ 3.  4.]]
h shape: (2, 3)
h is: 
[[ 0.  1.  2.]
 [ 1.  3.  4.]]
h2 shape: (2, 1, 3)
first element of h3 shape: (1, 3)
first element of h3 is: 
[[ 0.  1.  2.]]
h4 shape: (2, 3, 3)
[[[  0.   0.   0.]
  [  0.   1.   2.]
  [  0.   2.   4.]]

 [[  1.   3.   4.]
  [  3.   9.  12.]
  [  4.  12.  16.]]]
h5 shape: (2, 9)
[[  0.   0.   0.   0.   1.   2.   0.   2.   4.]
 [  1.   3.   4.   3.   9.  12.   4.  12.  16.]]


In [3]:
# In practice however the situation is more complicated,
# because TF does not infer the batch size and hence cannot
# use tf.pack and tf.unpack. Instead we use TensorArray, as follows

batch_size = 2
TensorArr = tf.TensorArray(tf.float32, 1, dynamic_size=True, infer_shape=False)
h3 = TensorArr.unpack(h2)

print("h2 shape: " + str(h2.get_shape()))
print("h3 size: " + str(h3.size()))
print("h3 first element: ")
print(sess.run(h3.read(0)))

h3p = tf.TensorArray(tf.float32, 1, dynamic_size=True,infer_shape=False)
for row in range(batch_size):
    row_tensor = h3.read(row)
    
    # Note it's important to put h3p = h3p.write each time
    h3p = h3p.write(row, tf.matmul(tf.transpose(row_tensor),row_tensor))

h4 = h3p.pack()
h5 = tf.reshape(h4,[2,-1]) # convert back to a 2D tensor

print("h5 shape: " + str(h5.get_shape()))
print(sess.run(h5))

h2 shape: (2, 1, 3)
h3 size: Tensor("TensorArraySize:0", shape=(), dtype=int32)
h3 first element: 
[[ 0.  1.  2.]]
h5 shape: (2, ?)
[[  0.   0.   0.   0.   1.   2.   0.   2.   4.]
 [  1.   3.   4.   3.   9.  12.   4.  12.  16.]]


In [4]:
# Alternatively we can use map_fn. Much better! Uses TensorArray under the hood
f = lambda x: tf.reshape(tf.matmul(tf.transpose(x),x),[-1])
h5 = tf.map_fn(f, h2)
print("h5 shape: " + str(h5.get_shape()))
print(sess.run(h5))

h5 shape: (2, 9)
[[  0.   0.   0.   0.   1.   2.   0.   2.   4.]
 [  1.   3.   4.   3.   9.  12.   4.  12.  16.]]


In [6]:
# Trying to understand Tensor / np.array operations
c = tf.constant([[1.0, 2.0], [3.0, 4.0]])
b = tf.constant([-1.0, 2.0])
print(sess.run(c+b))

[[ 0.  4.]
 [ 2.  6.]]


[array([ 0.,  1.]), array([ 0.,  1.]), array([ 0.,  1.]), array([ 0.,  1.]), array([ 1.,  0.]), array([ 1.,  0.]), array([ 0.,  1.]), array([ 0.,  1.]), array([ 0.,  1.]), array([ 0.,  1.]), array([ 0.,  1.]), array([ 0.,  1.])]


[0, 0]

[0, 1, 2]

SyntaxError: invalid syntax (<ipython-input-10-27ff79e93549>, line 1)

In [19]:
def f_reverse(seq):
    t = [0]*len(seq)
    for j in range(len(seq)):
        t[len(seq)-j-1] = seq[j]
    return t

seq = [0,1,0,1,1]

f_reverse(seq)

[1, 1, 0, 1, 0]

In [20]:
int(10.3)

10

In [21]:
3**2

9

In [39]:
c = tf.expand_dims(tf.constant([[1.0, 2.0], [3.0, 4.0]]),1)
print(sess.run(c))
print(c.get_shape())
d = tf.constant([[[1.0, 0.0], [0.0, 1.0]], [[2.0, 0.0], [0.0, 2.0]]])
e = tf.batch_matmul(c,d)
f = tf.squeeze(e)
print(sess.run(e))
print("")
print(sess.run(tf.reshape(e,[2,2])))

[[[ 1.  2.]]

 [[ 3.  4.]]]
(2, 1, 2)
[[[ 1.  2.]]

 [[ 6.  8.]]]

[[ 1.  2.]
 [ 6.  8.]]


In [8]:
operator_size = 2
state_size = 4

iota_matrix = []
for i in range(state_size):
    a = [0.0]*operator_size
    if i < operator_size:
        a[i] = 1.0
    iota_matrix.append(a)
    
rho_matrix = []
for i in range(operator_size):
    a = [0.0]*state_size
    if i < state_size:
        a[i] = 1.0
    rho_matrix.append(a)
    
print(iota_matrix)
print(rho_matrix)

[[1.0, 0.0], [0.0, 1.0], [0.0, 0.0], [0.0, 0.0]]
[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0]]
